<img src="./img/HWNI_logo.svg"/>

# Lab - One-Way ANOVA

In [ ]:
# makes our plots show up inside Jupyter
%matplotlib inline

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import scipy.stats

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

import util.utils as utils
import util.shared as shared

shared.format_plots()
shared.format_dataframes()

## About the Dataset

For this week's lab, we'll be using some EEG data graciously provided by the [Voytek lab](http://voyteklab.com/about-us/) of UCSD. Participants of varying ages were asked to perform a working memory task with varying levels of difficulty. The raw EEG signal has been summarized into the following two measures:

* [Contralateral Delay Activity](https://www.ncbi.nlm.nih.gov/pubmed/26802451), or CDA, is used to measure the engagement of visual working memory.

* [Frontal Midline Theta](https://www.ncbi.nlm.nih.gov/pubmed/9895201) oscillation amplitude has been correlated with sustained, internally-directed cognitive activity.

The performance of the subjects has also been summarized using the measure
[d'](https://en.wikipedia.org/wiki/Sensitivity_index) (pronounced "d-prime"), also known as the *sensitivity index*. D' is a measure of the subject's performance in  a task. It's based on comparing the true positive rate and false positive rate.

## Loading the Data

First, we load the data and take a look at a sample.

In [ ]:
df = pd.read_csv('./data/voytek_working_memory_aging_split.csv',index_col=None)

df.sample(5)

For the purposes of this lab, we're interested only in how task difficulty affects our three measures. We're uninterested in the subject's metadata -- `age_split`, `group`, `age`, and `idx`. Let's begin by dropping those columns from our dataframe using the DataFrame method `drop`.

In [ ]:
data = df.drop(['age_split','group','age','idx'], axis=1)
data[data.id == 1]

It's good practice to keep an original copy of your dataframe around (here, named `df`) so you can undo irreversible changes, like dropping columns.

If we're interested in looking at subject-by-subject information, we're not quite done with formatting our data. For subject-level analysis, our data is not yet tidy, since a single subject's observations are scattered over multiple rows. Use the `pivot` method to tidy our data. Hint: we want to get our row `index`es from the subject's `id` and to make new `columns` for our measures using the level of `difficulty`. Look back at the last lab (Lab B) for an example of using `pivot`.

In [ ]:
pivot_data = data.pivot(index='id', columns='difficulty')

In [ ]:
pivot_data.sample(5)

It now takes two pieces of information to pick out a particular column: the measure we're interested in (one of `d`, `cda`, or `fmt`) and the `difficulty` level: `1`, `2`, or `3`. These need to be provided in order.

In [ ]:
measure = 'cda'
difficulty = 2

pivot_data[measure, difficulty].head()

If we instead only index into the first level, we get a single-level dataframe back:

In [ ]:
pivot_data[measure].head()

## Visualizing the Data

Below, we'll be running ANOVA using difficulty as an independent factor and the three measures as dependent factors. Choose a visualization or collection of visualizations that you think would be appropriate for this purpose and plot the data below. Be ready to explain your choice in class.

You can use either the `pivot`ed data frame or the un`pivot`ed data frame.

Hint: the easiest way to apply most of our visualizations to this dataset is to write a loop that runs over our measures and produces a separate plot for each one.

Template:
```python
for measure in ["cda","fmt","d"]:
    plt.figure()
    data = pivot_data[measure]
    sns.?
    ...
    plt.title(measure+" versus difficulty")
```

Recall the assumptions behind ANOVA.

#### Q1 Based off of your visualization, do you think any of the assumptions of ANOVA are being violated for the case of fmt? What about d'?

## Built-In Tools for ANOVA

### Using `scipy`

First, we'll use the built-in `scipy.stats` function `f_oneway` to perform One-Way ANOVA.

In [ ]:
scipy.stats.f_oneway?

Try to organize the code you write in the cell below so that you minimize the amount you repeat yourself.

#### Follow-Up $t$-Tests

Select a measure with a significant ANOVA result and take a look at the data as you plotted it above.

It's often tempting to conclude on the basis of the ANOVA result
that any pattern we see or don't see in the data must be "real",
i.e. reflect a statistically significant result.

For example, on the measure `cda`,
we might be tempted to conclude that difficulty levels 2 and 3
are significantly different from difficulty level 1.

But a significant ANOVA result does not make a statement about
which levels of the factor are different from each other;
instead, it only says that the levels are somehow different.

To determine where the effect is coming from,
we need to perform follow-up $t$-tests
where we compare the means of two factor levels directly.
These tests will help us determine which inter-group differences are statistically significant.

In the cell below,
use `scipy.stats.ttest_ind`
(which we learned about in the two-sample testing lab)
to perform follow-up $t$-tests comparing
the contralateral delay activity, `cda`,
across difficulty levels -- 1 vs 2, 2 vs 3, and 1 vs 3.

#### Q2 What are your results? Are they different from what you expected?

You may be wondering what the point of the initial ANOVA was,
given that we needed to run additional statistical testing in order to determine our results.
Why not just go straight to the $t$-tests?

As mentioned in the tutorial for this section,
ANOVA first tests the null hypothesis that
all of the groups means are the same as the grand mean.
If we reject this null hypothesis only if $p < \alpha$,
then we know that the familywise error rate
for our follow-up $t$-tests can't be greater than $\alpha$,
since, if the null hypothesis is true,
we only perform them with probability $\alpha$.

Put less technically,
the initial ANOVA provides "cover" for our follow-up $t$-tests,
eliminating our multiple comparisons problem
by controlling the familywise error rate for the combined statistical test "ANOVA plus $t$-tests".

### Using `statsmodels`

In [ ]:
import statsmodels.api as sm

[`statsmodels`](http://www.statsmodels.org/stable/index.html)
is a Python package that implements
a number of popular features from the statistical computing language
[R](https://www.r-project.org/about.html).

Before we dive into how to use statsmodels,
we need to note that there are two equivalent ways of viewing the ANOVA.
In one view,
the ANOVA is a statistical test related to the $t$-test
but for more complicated experiments.
In the other view,
the ANOVA is a test of how well the data is explained
by the implicit linear model.
We have focused on the former view of ANOVA.

While there are interfaces for running ANOVA in R
that use the framework we've focused on,
there are not any stable,
mature packages for Python that implement
ANOVA in this way.

Instead,
these packages,
including statsmodels,
implement ANOVA as a two step process:

1. Fit the implicit linear model
1. Test the goodness-of-fit of that model.

Models are described,
as they are in R packages that use the same framework,
using *formula strings*.
These are short strings that specify
which columns of a dataframe are observations
and which columns are factors.

The formula functionality
of statsmodels
is complex enough that it
is imported separately
from the rest of statsmodels,
just like `scipy.stats`
is a separate import from `scipy`.

In [ ]:
import statsmodels.formula.api as smf

For a One-Way ANOVA,
the formula string is simple:

`'observation ~ factor'`

with one extra complication.
These formulas can describe models with mixtures of
categorical variables,
like our `difficulty` variable,
and numerical variables,
like our `age` variable.
If a categorical variable is coded as a number,
like our `difficulty` variable,
then we have to tell statsmodels
that the numbers in that variable should be treated as
category labels.
We do this by putting the variable's name inside a
`C()`,
like so:

`'observation ~ C(categorical_factor)'`

The statsmodels approach may seem like a lot of extra work,
especially compared to the scipy option.
However, this extra complexity is what
allows statsmodels to represent very complicated ANOVAs.

The cell below runs a One-Way ANOVA for the `cda` measure.
Compare the results to the output of `f_oneway` for this measure,
then change the formula to get the results for each of the other two measures.

In [ ]:
formula = 'cda ~ C(difficulty)'

ols_lm = smf.ols(formula, data=data)

fit = ols_lm.fit()

table = sm.stats.anova_lm(fit, typ=2)

table

Many folks find it easier to work with ANOVAs
in the formulation we used previously,
ANOVA as a hypothesis test,
rather than this new formulation,
ANOVA as a goodness-of-fit test for a model.
If that's the case for you,
and you need to run fairly complex ANOVAs,
then Python is not a good choice for this problem.

There are numerous packages for the R language
that implement ANOVA in different ways.
To learn more about R,
check out the awesome tutorials on
[R-tutor](http://www.r-tutor.com/).

But that doesn't mean you need to completely drop Python!
It's possible to
[run R scripts from inside Python](https://www.r-bloggers.com/integrating-python-and-r-part-ii-executing-r-from-python-and-vice-versa/),
which allows you to keep the rest of your analysis and visualization work in Python,
while running ANOVAs in R.
We won't go into more detail about this approach in this course.

## Finishing Up ANOVA by Hand

In the
[tutorial](./Tutorial - ANOVA by Hand.ipynb),
we walked through how to calculate the $F$-statistic by hand.
The code from that lab is conveniently wrapped up in the `utils` as
`anova_by_hand`, which returns the
`anova_frame`, the three dictionaries we used to calculate the $F$ value,
and $F$ itself.

Now, we'll add on the last few steps of an ANOVA:
determining the $p$-value
and reporting effect size with $\eta^2$ and $\omega^2$.

In [ ]:
measure = "cda"

anova_frame, sum_of_squares, dof, mean_square, F  = utils.anova_by_hand(data, measure)

In [ ]:
anova_frame.head(10)

### Calculating $p$

To interpret this $F$ value and so determine if the amount of variance we were able to explain is close to what one would expect by chance, we need to get a hold of the sampling distribution of the $F$-statistic under the null hypothesis. Once we have that, we can calculate the area under the curve from the observed value on up and use that to determine our $p$-value.

#### Q3 Why do we only calculate a "one-tailed" area, above our value, rather than a "two-tailed" area? 

We have two options for acquiring the sampling distribution of $F$:

1. Use a pre-calculated distribution. In the old days, that'd mean looking up values in a table in a reference text. These days, statistical computing libraries like `scipy` provide this service via functions.
1. Simulate what our data would look like under the null hypothesis and use the distribution of the $F$-statistic from our simulations to approximate the true distribution of $F$

We'll take the second road.
This will involve a
[*resampling*](https://en.wikipedia.org/wiki/Resampling_%28statistics%29) technique called an
[*approximate permutation test*](https://en.wikipedia.org/wiki/Resampling_%28statistics%29#Monte_Carlo_testing).
It is closely related to
[exact tests](https://en.wikipedia.org/wiki/Exact_test).

We discussed approximate permutation tests in the section on hypothesis testing.

#### Q4 Explain, in your own words, what the null hypothesis of the ANOVA test is.

#### Q5 Under the null hypothesis of ANOVA, what can we say about the relationship between the group label and the measurement? 

#### Q6 Based on this relationship, how could we simulate what our data would look like under the null hypothesis? That is, how could we generate data that is distributed according to the null hypothesis?

The function `utils.estimate_f_distribution` will implement this simulation and then compare the results to the true null sampling distribution of $F$, provided by `scipy`.

In [ ]:
measure = "cda"
groups = data.difficulty.unique()

grouped_data = utils.group_data(pivot_data,measure,groups)

Fs = utils.estimate_F_distribution(grouped_data)

sorted_Fs = sorted(Fs)

In [ ]:
plt.figure(figsize=(12,4))
utils.plot_approximated_F(Fs)
utils.plot_true_F(dof["explained"], dof["residual"])

We can calculate an approximate $p$-value for the value of $F$ calculated above by subtracting one from the empirical cumulative density function of the `sorted_Fs` evaluated at the observed value.

#### Q7 Explain why subtracting one from the empirical CDF of the statistic under the null distribution evaluated at the observed value gives the $p$-value.

The cell below checks that the answers given by this method and by `scipy` are in agreement.

In [ ]:
def eCDF(F, sorted_Fs):
    try:
        index = np.where(sorted_Fs > F)[0][0] #returns a tuple of arrays, we want first element of first array
    except IndexError:
        index = len(sorted_Fs)
        
    return index/len(sorted_Fs)

approx_p = 1-eCDF(F, sorted_Fs)
       
_, scipy_p = run_ANOVA(pivot_data, measure)
       
(scipy_p - approx_p)**2

As part of the process of generating our estimated $F$ distribution, we need to generate a bunch of data sets that are distributed according to the null hypothesis. We can use these, plus any method that calculates $p$-values, to simulate the distribution of $p$-values under the null hypothesis. The function `utils.simulate_null` will do this for us.

In [ ]:
ps = utils.simulate_null(grouped_data, N=10000)

#### Q8 What sampling distribution do we expect the $p$-value to have under the null hypothesis? How might this distribution look different under the alternative hypothesis?

In the cell below, plot the simulated sampling distribution of the $p$-value. You'll want `sns.distplot`.

In [ ]:
sns.distplot(ps, bins=np.arange(-0.25,1.3,0.05), kde=False,
             hist_kws={'histtype':'stepfilled',
                       'density':True,
                       'linewidth':4,
                       'edgecolor':'gray',
                       'alpha':0.8}, label =r"Null Distribution of $p$");
plt.ylim([0, 1.5])
plt.legend();

### Beyond $p$ - Effect Size Reporting with $\eta^2$ and $\omega^2$

The $F$-statistic is used to determine the statistical significance of an ANOVA result.

Recall that previously, we've distinguished between practical/scientific and statistical significance. Statistical significance is centered around whether an effect exists at all.

#### Q9 How is this different from the practical or scientific significance of an ANOVA result?

The APA recommends that, in addition to reporting $F$ and $p$, scientists report the value $\eta^2$, which is equal to the ratio of the explained sum-of-squares to the explainable sum-of-squares. It's also known as the *variance explained*.

#### Q10 Why is this number closer to a notion of practical significance than $F$ is?

Calculate $\eta^2$ for a statistically significant test you ran above.

#### Q11 Does this seem like a practically significant fraction to you? Look back at the visualizations of the data you produced at the beginning of the lab. Are there any visual hints that would lead you to expect a value of $\eta^2$ close to what you calculated?

Notice that $\eta^2$ uses the sums-of-squares, rather than mean squares.
This means that it doesn't take into account degrees of freedom
and is therefore a biased estimator of the quantity it is trying to capture:
it overestimates how much variance has been explained.
Intuitively, it only captures how well you explained your dataset,
not how well you might explain additional data from the same population.

An unbiased estimator for explained variance exists, known as $\omega^2$ (pronounced "omega-squared"). You can [read more about it here](http://daniellakens.blogspot.com/2015/06/why-you-should-use-omega-squared.html).

Use the following formula to compute $\omega^2$ for your test:

$$
    \omega^2 = \frac{F-1}{\frac{F+1+\text{df}_{residual}}{\text{df}_{explained}}}
$$

Template:
```python
om_sqrd = ?/(?/?)
```

#### Q12 Does this more accurate estimate change your opinion of the the practical significance of the results of your test?